In [1]:
from pymongo import MongoClient
import pandas as pd
import os

from dotenv import load_dotenv


load_dotenv()

client = MongoClient(os.getenv("MONGODB_URI"))
db = client[os.getenv("DB_NAME")]

# Load collection 'csv1_data' $ 'csv2_data' into a Pandas DataFrame
df2_test = pd.DataFrame(list(db["csv1_data"].find({}, {"_id": 0})))
df1_train = pd.DataFrame(list(db["csv2_data"].find({}, {"_id": 0})))
df1_train.head()




,id,SMILES,Tm,Group 1,Group 2,Group 3,Group 4,Group 5,Group 6,Group 7,...,Group 415,Group 416,Group 417,Group 418,Group 419,Group 420,Group 421,Group 422,Group 423,Group 424
0,2175,FC1=C(F)C(F)(F)C1(F)F,213.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1222,c1ccc2c(c1)ccc3Nc4ccccc4c23,407.15,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2994,CCN1C(C)=Nc2ccccc12,324.15,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1704,CC#CC(=O)O,351.15,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2526,CCCCC(S)C,126.15,2,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [2]:
df2_test.head()

,id,SMILES,Group 1,Group 2,Group 3,Group 4,Group 5,Group 6,Group 7,Group 8,...,Group 415,Group 416,Group 417,Group 418,Group 419,Group 420,Group 421,Group 422,Group 423,Group 424
0,1022,CCOC(=O)c1ccc(O)cc1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1146,CCCCCCc1ccc(O)cc1O,1,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,79,ClCBr,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2279,C=CCCCCCCCC,1,7,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1342,Fc1ccc(cc1)C(F)(F)F,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
def load_data():
    df1_train = pd.DataFrame(list(db["csv2_data"].find({}, {"_id": 0})))
    df2_test = pd.DataFrame(list(db["csv1_data"].find({}, {"_id": 0})))
   

    print(f'Target stats: mean = {df1_train['Tm'].mean():.2f},std={df1_train['Tm'].std():.2f}')
    print(f'Target range: [{df1_train['Tm'].min():.1f}, {df1_train['Tm'].max():.1f}]')

    return df1_train, df2_test

In [4]:
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

from rdkit import Chem, RDLogger
from rdkit.Chem import Descriptors, rdMolDescriptors, Crippen, Lipinski, Fragments, AllChem
from rdkit.Chem import rdFingerprintGenerator,MACCSkeys
from rdkit.Chem.AllChem import ComputeGasteigerCharges

from sklearn.metrics import mean_absolute_error 
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold

from lightgbm import LGBMRegressor
import lightgbm as lgb
from xgboost import XGBRegressor
from xgboost import callback
from catboost import CatBoostRegressor

RDLogger.DisableLog('rdApp.*')




In [5]:
#Feature Enginnering 
class MolecularFeatures:
    '''
    Extract rich molecular feature from SMILES
    '''
    def __init__(self, morgan_bits = 1024, morgan_radius=3):
        self.morgan_bits = morgan_bits
        self.morgan_radius = morgan_radius

    def safe_calc (self, func, mol, default = 0):
        try:
            val = func(mol)
            return default if pd.isna(val) or np.isinf(val) else val
        except:
            return default
    
    def extract_one(self, smiles):
        '''
        Extract all features for one smiles
        '''
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return{}
        
        features = {}

        # EDkit Descriptors(~200 features)

        for name, func in Descriptors._descList:
            features[name] = self.safe_calc(func,mol)

        
        #Key molecular properties
        features['MoLLogp'] = self.safe_calc(Crippen.MolLogP, mol)
     
        
        features['MolMR'] = self.safe_calc(Crippen.MolMR, mol)
        features['TPSA'] = self.safe_calc(rdMolDescriptors.CalcTPSA, mol)
        features['LabuteASA'] = self.safe_calc(rdMolDescriptors.CalcLabuteASA, mol)
        features['NumHAcceptors'] = self.safe_calc(Lipinski.NumHAcceptors, mol)
        features['NumHDonors'] = self.safe_calc(Lipinski.NumHDonors, mol)
        features['NumRotatableBonds'] = self.safe_calc(Lipinski.NumRotatableBonds, mol)
        features['NumHeteroatoms'] = self.safe_calc(Lipinski.NumHeteroatoms, mol)
        
        # === 3) Ring Statistics ===
        features['NumRings'] = self.safe_calc(rdMolDescriptors.CalcNumRings, mol)
        features['NumAromaticRings'] = self.safe_calc(rdMolDescriptors.CalcNumAromaticRings, mol)
        features['NumAliphaticRings'] = self.safe_calc(rdMolDescriptors.CalcNumAliphaticRings, mol)
        features['NumSaturatedRings'] = self.safe_calc(rdMolDescriptors.CalcNumSaturatedRings, mol)
        features['NumHeterocycles'] = self.safe_calc(rdMolDescriptors.CalcNumHeterocycles, mol)
        features['NumAromaticHeterocycles'] = self.safe_calc(rdMolDescriptors.CalcNumAromaticHeterocycles, mol)
        
        # === 4) Complexity & Shape ===
        features['BertzCT'] = self.safe_calc(Descriptors.BertzCT, mol)

        # features['BertzCT'] = self.safe_calc(rdMolDescriptors.CalcBertzCT, mol)
        features['NumBridgeheadAtoms'] = self.safe_calc(rdMolDescriptors.CalcNumBridgeheadAtoms, mol)
        features['NumSpiroAtoms'] = self.safe_calc(rdMolDescriptors.CalcNumSpiroAtoms, mol)

        # Functional Groups(~80 features)
        for attr in dir(Fragments):
            if attr.startswith('fr_'):
                features[attr] = self.safe_calc(getattr(Fragments, attr),mol)
            
        #Morgan Fingerprint(1024 bits)
        try:
            mgen = rdFingerprintGenerator.GetMorganGenerator(
                radius= self.morgan_radius,
                fpSize=self.morgan_bits
            )

            mfp = mgen.GetFingerprint(mol)
            for i in range(self.morgan_bits):
                features[f'Morgan_{i}'] = int(mfp[i])
        except:
            for i in range(self.morgan_bits):
                features[f'Morgan_{i}'] = 0

        # MACCS keys(167 structural patterns)
        try:
            maccs = MACCSkeys.GenMACCSKeys(mol)
            for i in range(len(maccs)):
                features[f'MACCS_{i}'] = int(maccs[i])
        except:
            for i in range(167):
                features[f'MACCS_{i}'] = 0 


        # === 8) 3D Shape Features (important for melting point!) ===
        try:
            mol_h = Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol_h, randomSeed=42)
            AllChem.UFFOptimizeMolecule(mol_h, maxIters=200)
            mol_clean = Chem.RemoveHs(mol_h)
            
            features['PMI1'] = self.safe_calc(lambda m: rdMolDescriptors.CalcPMI1(m, 0), mol_clean)
            features['PMI2'] = self.safe_calc(lambda m: rdMolDescriptors.CalcPMI2(m, 0), mol_clean)
            features['PMI3'] = self.safe_calc(lambda m: rdMolDescriptors.CalcPMI3(m, 0), mol_clean)
            features['NPR1'] = self.safe_calc(lambda m: rdMolDescriptors.CalcNPR1(m, 0), mol_clean)
            features['NPR2'] = self.safe_calc(lambda m: rdMolDescriptors.CalcNPR2(m, 0), mol_clean)
            features['RadiusOfGyration'] = self.safe_calc(
                lambda m: rdMolDescriptors.CalcRadiusOfGyration(m, 0), mol_clean
            )
            features['InertialShapeFactor'] = self.safe_calc(
                lambda m: rdMolDescriptors.CalcInertialShapeFactor(m, 0), mol_clean
            )
            features['Eccentricity'] = self.safe_calc(
                lambda m: rdMolDescriptors.CalcEccentricity(m, 0), mol_clean
            )
            features['Asphericity'] = self.safe_calc(
                lambda m: rdMolDescriptors.CalcAsphericity(m, 0), mol_clean
            )
            features['SpherocityIndex'] = self.safe_calc(
                lambda m: rdMolDescriptors.CalcSpherocityIndex(m, 0), mol_clean
            )
        except:
            pass
        
        #  Custom Chemistry Features ===
        # Atom counts
        for elem in ['C', 'N', 'O', 'S', 'F', 'Cl', 'Br', 'I', 'P']:
            features[f'Count_{elem}'] = sum(1 for a in mol.GetAtoms() if a.GetSymbol() == elem)
        
        # Charge features
        total_charge = sum(a.GetFormalCharge() for a in mol.GetAtoms())
        features['FormalCharge'] = total_charge
        has_pos = any(a.GetFormalCharge() > 0 for a in mol.GetAtoms())
        has_neg = any(a.GetFormalCharge() < 0 for a in mol.GetAtoms())
        features['IsZwitterion'] = int(has_pos and has_neg)
        
        # Hybridization counts
        features['SP3_Count'] = sum(1 for a in mol.GetAtoms() 
                                    if a.GetHybridization() == Chem.HybridizationType.SP3)
        features['SP2_Count'] = sum(1 for a in mol.GetAtoms() 
                                    if a.GetHybridization() == Chem.HybridizationType.SP2)
        features['SP_Count'] = sum(1 for a in mol.GetAtoms() 
                                   if a.GetHybridization() == Chem.HybridizationType.SP)
        
        # Bond type counts
        bonds = list(mol.GetBonds())
        features['SingleBonds'] = sum(1 for b in bonds if b.GetBondType() == Chem.BondType.SINGLE)
        features['DoubleBonds'] = sum(1 for b in bonds if b.GetBondType() == Chem.BondType.DOUBLE)
        features['TripleBonds'] = sum(1 for b in bonds if b.GetBondType() == Chem.BondType.TRIPLE)
        features['AromaticBonds'] = sum(1 for b in bonds if b.GetIsAromatic())
        
        # H-bonding potential (critical for melting point!)
        hbd = features.get('NumHDonors', 0)
        hba = features.get('NumHAcceptors', 0)
        features['HBond_Capacity'] = hbd + hba
        features['HBond_Product'] = hbd * hba
        features['HBond_Ratio'] = hbd / (hba + 1)
        
        # Polarity & symmetry proxies
        features['Polarity_Index'] = (hbd + hba) / (mol.GetNumAtoms() + 1)
        features['Aromatic_Fraction'] = features['NumAromaticRings'] / (features['NumRings'] + 1)
        
        # Gasteiger charges (electronic effects)
        try:
            mol_h = Chem.AddHs(mol)
            ComputeGasteigerCharges(mol_h)
            charges = [a.GetDoubleProp('_GasteigerCharge') for a in mol_h.GetAtoms() 
                      if a.HasProp('_GasteigerCharge')]
            charges = [c for c in charges if not (pd.isna(c) or np.isinf(c))]
            if charges:
                features['Gasteiger_Mean'] = np.mean(charges)
                features['Gasteiger_Std'] = np.std(charges)
                features['Gasteiger_Min'] = np.min(charges)
                features['Gasteiger_Max'] = np.max(charges)
                features['Gasteiger_Range'] = np.max(charges) - np.min(charges)
        except:
            pass
        
        return features
    
    def transform(self, df):
        """Transform entire dataframe"""
        print(f"\n🧬 Extracting features for {len(df)} molecules...")
        
        features_list = []
        for idx, row in df.iterrows():
            if idx > 0 and idx % 500 == 0:
                print(f"  Progress: {idx}/{len(df)}")
            features_list.append(self.extract_one(row['SMILES']))
        
        df_features = pd.DataFrame(features_list).fillna(0)
        
        # Remove constant columns
        nunique = df_features.nunique()
        constant_cols = nunique[nunique <= 1].index
        df_features = df_features.drop(columns=constant_cols)
        
        # Remove duplicate columns
        df_features = df_features.loc[:, ~df_features.T.duplicated()]
        
        print(f"  ✅ Extracted {df_features.shape[1]} features")

        return df_features
        
    
        

In [6]:
#Ensemble Model with K-fold cv
class EnsembleModel:
    def __init__(self,n_folds=10, random_state=7):
        self.n_folds = n_folds
        self.random_state = random_state
        self.models = {}
        self.scalers = []
        self.feature_name = None

    def get_base_model(self):
        return{
            'lgbm_default' : LGBMRegressor(
                n_estimators=5000,
                learning_rate= 0.015,
                num_leaves=64,
                max_depth=3,
                min_child_samples=18,
                subsample=0.7,
                colsample_bytree=0.7,
                reg_alpha=0.1,
                reg_lambda=0.1,
                random_state=self.random_state,
                verbose=-1
            ),
            'lgbm_deep': LGBMRegressor(
                n_estimators=1500,
                learning_rate=0.02,
                num_leaves=140,
                max_depth=10,
                min_child_samples=25,
                subsample=0.7,
                colsample_bytree=0.7,
                reg_alpha=1.0,
                reg_lambda=1.0,
                random_state=self.random_state + 3,
                verbose = -1
            ),
            'xgb_default': XGBRegressor(
                n_estimator = 1800,
                learning_rate=0.06,
                max_depth=9,
                min_child_weight=2,
                subsample=0.9,
                colsample_bytree=0.8,
                reg_alpha=0.1,
                reg_lambda=0.1,
                random_state=self.random_state,
                tree_method='hist',
                verbosity=0
            ),
            'catboost_default':CatBoostRegressor(
                iterations=2000,
                learning_rate=0.04,
                depth=8,
                l2_leaf_reg=4,
                random_state=self.random_state,
                verbose=0
            ),

        }
    
    def train(self, X, y):
        '''
        Train ensemble with k -fold cross-validation
        '''
        print('-'*60)
        print(f'Training ensemble ({self.n_folds} -fold cv)')
        print('-'*60)

        self.feature_names = X.columns.tolist()
        kf = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.random_state)

        base_models = self.get_base_model()
        oof_predictions = {name: np.zeros(len(X)) for name in base_models}
        
        #Initialize model storage
        for name in base_models:
            self.models[name] = []

        #K-fold training
        for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
            print(f'\n FOld {fold+1}/{self.n_folds}')
            print("-"*40)

            X_train, X_val = X.iloc[train_idx].copy(), X.iloc[val_idx].copy()
            y_train, y_val = y.iloc[train_idx].copy(), y.iloc[val_idx].copy()

            #feature scaling (RobustScaler handles outliers wll)
            scaler = RobustScaler()
            X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train),
                columns=X.columns,
                index=X_train.index

            )
            X_val_scaled = pd.DataFrame(
                scaler.transform(X_val),
                columns=X.columns,
                index=X_val.index
            )
            self.scalers.append(scaler)

            #Train each model
            for model_name, model in base_models.items():
                from sklearn.base import clone
                model_clone = clone(model)

                #Train with early stooping
                if 'lgbm' in model_name:
                    model_clone.fit(
                        X_train_scaled,y_train,
                        eval_set = [(X_val_scaled, y_val)],
                        callbacks = [lgb.early_stopping(stopping_rounds=100, verbose = False)]

                    )
                elif 'xgb' in model_name:
                    model_clone.fit(
                        X_train_scaled,y_train,
                        eval_set = [(X_val_scaled,y_val)],
                        early_stopping_rounds=100,
                        verbose=False
                    )
                elif 'catboost' in model_name:
                    model_clone.fit(
                        X_train_scaled,y_train,
                        eval_set =(X_val_scaled,y_val),
                        early_stopping_rounds = 100,
                        verbose = False
                    )
                else:
                    model_clone.fit(X_train_scaled,y_train)

                
                #out of fold prediction
                oof_pred = model_clone.predict(X_val_scaled)
                oof_predictions[model_name][val_idx] = oof_pred

                #Validation Score
                mae = mean_absolute_error(y_val,oof_pred)
                print(f'{model_name:18s} : MAE = {mae:.4f}')

                self.models[model_name].append(model_clone)

        #Calculate overall OOF scores 
        print('-'*45)
        print('OUT_OF_FOLD_SCORES(full training set)')
        print('-'*45)

        model_maes = {}
        for model_name, predictions in oof_predictions.items():
            mae = mean_absolute_error(y, predictions)
            model_maes[model_name] = mae
            print(f'{model_name:18s}: {mae:.4f}')

        #Ensembel (simple average)
        ensemble_pred = np.mean(list(oof_predictions.values()),axis = 0)
        ensemble_mae = mean_absolute_error(y, ensemble_pred)
        print(f'\n {'ENSEMBLE (Average)':18s}: {ensemble_mae:.4f}')
        print('-'*45)
        return oof_predictions, ensemble_mae
    
    def predict(self, X):
        '''
        Generate predictions with full ensemble
        '''
        all_fold_predictions = []
        
        #Get predictions from each fold
        for fold_idx, scaler in enumerate(self.scalers):
            X_scaled = pd.DataFrame(
                scaler.transform(X),
                columns=self.feature_names

            )
            #Get predictions from all models in this fold
            fold_model_preds =[]
            for model_name, models in self.models.items():
                pred = models[fold_idx].predict(X_scaled)
                fold_model_preds.append(pred)


            #AVerage models with fold
            fold_pred = np.mean(fold_model_preds,axis=0)
            all_fold_predictions.append(fold_pred)

        #Average across folds
        final_predictions = np.mean(all_fold_predictions,axis=0)

        print(f'Predicition range:[{final_predictions.min():.1f}, {final_predictions.max():.1f}]')
        print(f'Prediction mean: {final_predictions.mean():.1f}')
        return final_predictions

    

        

In [ ]:
def main():
    df1_train, df2_test = load_data()

    # Extract the provided Group features
    group_cols = [col for col in df1_train.columns if col.startswith('Group')]
    print(f"Found {len(group_cols)} provided Group features.")

    if len(group_cols) == 0:
        print("Warning: No 'Group' columns found! Check column names.")
    # ===============================================

    feature_engine = MolecularFeatures(morgan_bits=1024, morgan_radius=3)

    X_train = feature_engine.transform(df1_train)
    X_test = feature_engine.transform(df2_test)

    # === NEW: Combine RDKit features + Original Group features ===
    X_train_groups = df1_train[group_cols].reset_index(drop=True)
    X_test_groups = df2_test[group_cols].reset_index(drop=True)

    X_train = pd.concat([X_train, X_train_groups], axis=1)
    X_test = pd.concat([X_test, X_test_groups], axis=1)
    # ===============================================


    #Align features
    common_features = list(set(X_train.columns) & set(X_test.columns))
    X_train = X_train[common_features]
    X_test = X_test[common_features]

    y_train = pd.Series(df1_train['Tm'].values)

    print(f'\n Ready for training')

    print(f'X_train: {X_train.shape}')
    print(f'X_test: {X_test.shape}')
    print(f'y_train: {y_train.shape}')

    #train ensemble
    ensemble = EnsembleModel(n_folds=10, random_state=7)
    oof_preds, cv_score = ensemble.train(X_train,y_train)

    #Make Predictions
    predictions = ensemble.predict(X_test)

    #post pre-processing
    #melting point are typically 40-700k for orginic compunds
    predictions = np.clip(predictions, 50 , 700)

    #Create Submission
    submission = pd.DataFrame({

        'id':df2_test['id'],
        'Tm': predictions
    })
    submission = submission.drop_duplicates(subset='ID', keep='first')

    submission.to_csv('submission.csv', index=False)
    

    print(f'\n{'-' *60}')
    print(f'Row: {len(submission)}')
    print(f'Prediction range: [{predictions.min():.1f},  {predictions.max():.1f}]')

    print('\n Smaple submision')
    print(submission.head(7))

    return submission, cv_score





    

In [8]:
if __name__ == '__main__':
    submission, cv_score = main()
    print('Complete')


Target stats: mean = 278.26,std=85.11
Target range: [53.5, 897.1]
Found 424 provided Group features.

🧬 Extracting features for 5324 molecules...
  Progress: 500/5324
  Progress: 1000/5324
  Progress: 1500/5324
  Progress: 2000/5324
  Progress: 2500/5324
  Progress: 3000/5324
  Progress: 3500/5324
  Progress: 4000/5324
  Progress: 4500/5324
  Progress: 5000/5324
  ✅ Extracted 1403 features

🧬 Extracting features for 1332 molecules...
  Progress: 500/1332
  Progress: 1000/1332
  ✅ Extracted 1361 features

 Ready for training
X_train: (5324, 1783)
X_test: (1332, 1783)
y_train: (5324,)
------------------------------------------------------------
Training ensemble (10 -fold cv)
------------------------------------------------------------

 FOld 1/10
----------------------------------------
lgbm_default       : MAE = 20.3450
lgbm_deep          : MAE = 12.2380
xgb_default        : MAE = 11.8852
catboost_default   : MAE = 11.6375

 FOld 2/10
----------------------------------------
lgbm_defau